In [16]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
import qiskit.quantum_info as qi
import scipy.linalg as la

In [19]:
A = np.array([
    [1.5, 0.5],
    [0.5, 1.5],
    ]).astype('complex')
U = la.expm(1j*A)
eigs_u, vecs_u = np.linalg.eig(U)
print(vecs_u[1])

[-0.70710678+1.73472348e-16j  0.70710678+3.33066907e-16j]


In [3]:
#states
state_zero = np.array([[1.0],[0.0]])
state_one = np.array([[0.0],[1.0]])
#projectors
P0 = np.dot(state_zero, state_zero.T)
P1 = np.dot(state_one, state_one.T)

In [4]:
def multi_kron(*args):
    ret = np.array([[1.0]]).astype('complex')
    for q in args:
        ret = np.kron(ret, q)
    return ret

def multi_dot(*args):
    ret = np.eye(np.shape(args[0])[0]).astype('complex')
    for q in args:
        ret = np.dot(ret, q)
    return ret

In [223]:
# def U(A, phi):
#     return expm(-1j*A*phi).astype('complex')

def H():
    return 1/np.sqrt(2) * np.array([[1, 1],
                                    [1, -1]]).astype('complex')
def X():
    return np.array([[0, 1],
                     [1, 0]]).astype('complex')
def Rz(phi):
    return np.array([[np.exp(-1j*phi/2), 0],
                     [0, np.exp(1j*phi/2)]]).astype('complex')
def Ry(theta):
    return np.array([[np.cos(theta/2), -np.sin(theta/2)],
                     [np.sin(theta/2), np.cos(theta/2)]]).astype('complex')
def I():
    return np.array([[1, 0],
                     [0, 1]]).astype('complex')

#for some reson CONTROL and TARGET are changed
def control(control_q, target_q, n_of_qubits, operator):
    list_of_ops_left = []
    list_of_ops_right = []
    for i in range(n_of_qubits):
        if i == control_q:
            list_of_ops_left.append(P0)
            list_of_ops_right.append(P1)
        elif i == target_q:
            list_of_ops_left.append(I())
            list_of_ops_right.append(operator)
        else:
            list_of_ops_left.append(I())
            list_of_ops_right.append(I())
    return multi_kron(*list_of_ops_left) + multi_kron(*list_of_ops_right).astype('complex')

In [224]:
prime_state = multi_kron(np.array([[el] for el in vecs_u[1]]), state_zero, state_zero,
                         state_zero)
# prime_state = multi_kron(state_zero, state_zero, state_zero,
#                          state_zero)
# print(prime_state)

In [225]:
#state preparation
state = multi_kron(I(), H(), H(), H())
# prime_state = np.dot(state, prime_state)
# print(prime_state)

In [226]:

#eigenvalues to phases
etp_one = control(control_q=1,
                  target_q=0,
                  n_of_qubits=4,
                  operator=U**1) #4
etp_two = control(control_q=2,
                  target_q=0,
                  n_of_qubits=4,
                  operator=U**1) #2
etp_three = control(control_q=3,
                  target_q=0,
                  n_of_qubits=4,
                  operator=U**1) #1
# etp_four = control(control_q=3,
#                   target_q=4,
#                   n_of_qubits=4,
#                   operator=U**8)

etp_u = multi_dot(etp_three, etp_two, etp_two, etp_one, etp_one, etp_one, etp_one)

# prime_state = np.dot(final_u, prime_state)
# print(final_u.shape)
# print(final_u.round(3))
# print(U)

In [323]:
# #inverse_fourier
if_one = multi_kron(I(), H(), I(), I())
if_two = control(
    control_q=1,
    target_q=0,
    n_of_qubits=2,
    operator=Rz(np.pi/2))

# if_three = multi_kron(I(), I(), H(), I())
# if_four = control(control_q=1,
#                   target_q=3,
#                   n_of_qubits=4,
#                   operator=Rz(-np.pi))
# if_five = control(control_q=1,
#                   target_q=2,
#                   n_of_qubits=4,
#                   operator=Rz(-np.pi))
# if_six = multi_kron(I(), H(), I(), I())

# etp_ifft = multi_dot(if_six, if_five, if_four, if_three, if_two, if_one)
# etp_ifft = multi_dot(if_three, if_two, if_one)

print(etp_ifft.shape)
print(if_two.round(3))

(16, 16)
[[1.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j   ]
 [0.   +0.j    0.707-0.707j 0.   +0.j    0.   +0.j   ]
 [0.   +0.j    0.   +0.j    1.   +0.j    0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.   +0.j    0.707+0.707j]]


In [128]:


# print(np.round(multi_dot(state, etp_one, etp_two, etp_three), 3))
#Fourier transform
# ft_one = multi_kron(I(), H(), I(), I(), I())
# ft_two = control(control_q=1,
#                  target_q=3,
#                  n_of_qubits=5,
#                  operator=Rz(-np.pi/4))
# ft_three = control(control_q=1,
#                    target_q=2,
#                    n_of_qubits=5,
#                    operator=Rz(-np.pi/2))
# ft_four = multi_kron(I(), I(), H(), I(), I())
# ft_five = control(control_q=2,
#                   target_q=3,
#                   n_of_qubits=5,
#                   operator=Rz(-np.pi/2))
# ft_six = multi_kron(I(), I(), I(), H(), I())
# print(np.round(multi_dot(state,
#                          etp_one, etp_two, etp_three,
#                          ft_one, ft_two, ft_three, ft_four, ft_five, ft_six), 3))
'''
# #Ancilla qubit
aq_one = control(control_q=1,
                 target_q=0,
                 n_of_qubits=5,
                 operator=Ry(np.pi/2)) 
aq_two = control(control_q=2,
                 target_q=0,
                 n_of_qubits=5,
                 operator=Ry(np.pi/4))
aq_three = control(control_q=3,
                   target_q=0,
                   n_of_qubits=5,
                   operator=Ry(np.pi/8)) 
# print(np.round(multi_dot(
#     aq_three, aq_two, aq_one,
#     ft_six, ft_five, ft_four, ft_three, ft_two, ft_one,
#     etp_three, etp_two, etp_one,
#     state), 3))
'''

# print(np.round(multi_dot(
#     if_seven, if_six, if_five, if_four, if_three, if_two, if_one,
#     aq_three, aq_two, aq_one,
#     ft_six, ft_five, ft_four, ft_three, ft_two, ft_one,
#     etp_three, etp_two, etp_one,
#     state), 3))
'''
# #phases to eigenvalues
pte_one = control(control_q=1,
                  target_q=4,
                  n_of_qubits=5,
                  operator=U(A, -np.pi/8))
pte_two = control(control_q=2,
                  target_q=4,
                  n_of_qubits=5,
                  operator=U(A, -np.pi/4))
pte_three = control(control_q=3,
                    target_q=4,
                    n_of_qubits=5,
                    operator=U(A, -np.pi/2))

final_u = multi_dot(
    if_seven, if_six, if_five, if_four, if_three, if_two, if_one,
    etp_three, etp_two, etp_one,
    state)
    '''
# print(np.round(final_u, 3))


(16, 16)
[[ 0.354+0.j     0.135-0.327j  0.25 -0.25j  -0.135-0.327j  0.354+0.j
   0.135-0.327j  0.25 -0.25j  -0.135-0.327j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [ 0.354+0.j    -0.135+0.327j  0.25 -0.25j   0.135+0.327j  0.354+0.j
  -0.135+0.327j  0.25 -0.25j   0.135+0.327j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [ 0.354+0.j     0.327+0.135j -0.25 +0.25j  -0.327+0.135j  0.354+0.j
   0.327+0.135j -0.25 +0.25j  -0.327+0.135j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [ 0.354+0.j    -0.327-0.135j -0.25 +0.25j   0.327-0.135j  0.354+0.j
  -0.327-0.135j -0.25 +0.25j   0.327-0.135j  0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j   ]
 [ 0.354+0.j     0.327-0.135j  0.25 +0.25j   0.327+0.135j -0.354+0.j
  -0.327+0.135j -0.25 -0.2

'\n# #phases to eigenvalues\npte_one = control(control_q=1,\n                  target_q=4,\n                  n_of_qubits=5,\n                  operator=U(A, -np.pi/8))\npte_two = control(control_q=2,\n                  target_q=4,\n                  n_of_qubits=5,\n                  operator=U(A, -np.pi/4))\npte_three = control(control_q=3,\n                    target_q=4,\n                    n_of_qubits=5,\n                    operator=U(A, -np.pi/2))\n\nfinal_u = multi_dot(\n    if_seven, if_six, if_five, if_four, if_three, if_two, if_one,\n    etp_three, etp_two, etp_one,\n    state)\n    '

In [7]:
prime_state = multi_kron(state_zero, state_zero, state_zero, vecs_u[1])
final_state = np.dot(final_u, prime_state)
rho = np.dot(final_state, np.conj(final_state).T)
print(np.round(rho, 5), '\n')

[[ 0.12711+0.j       0.05814+0.05646j -0.13596+0.08869j ...
   0.     +0.j       0.     +0.j       0.     +0.j     ]
 [ 0.05814-0.05646j  0.05167-0.j      -0.02279+0.10095j ...
   0.     +0.j       0.     +0.j       0.     +0.j     ]
 [-0.13596-0.08869j -0.02279-0.10095j  0.20731+0.j      ...
   0.     +0.j       0.     +0.j       0.     +0.j     ]
 ...
 [ 0.     +0.j       0.     +0.j       0.     +0.j      ...
   0.     +0.j       0.     +0.j       0.     +0.j     ]
 [ 0.     +0.j       0.     +0.j       0.     +0.j      ...
   0.     +0.j       0.     +0.j       0.     +0.j     ]
 [ 0.     +0.j       0.     +0.j       0.     +0.j      ...
   0.     +0.j       0.     +0.j       0.     +0.j     ]] 



In [8]:
proj = multi_kron(P0, I(), I(), I(), P1)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 3))
proj = multi_kron(P0, I(), I(), I(), P0)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 3))
proj = multi_kron(P1, I(), I(), I(), P1)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 3))
proj = multi_kron(P1, I(), I(), I(), P0)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 3))

(0.437-0j)
(0.562+0j)
0j
0j


In [9]:
prob0 = np.trace(np.dot(multi_kron(I(), I(), I(), I(), P1), rho))
prob0

(0.4374999999999996-2.977195965426708e-18j)